<a href="https://colab.research.google.com/github/jeffvun/Machine-Learning-Labs/blob/main/Image_Forensics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import Libraries**

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import FasterRCNN, ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model


## **Data Preprocessing**

In [ ]:
# Define your data directories
data_dir = '/Content/data'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

# Image data generator for data augmentation
datagen = ImageDataGenerator(rescale=1./255,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True)

# Create generators for training and testing data
batch_size = 32
train_generator = datagen.flow_from_directory(train_dir,
                                              target_size=(224, 224),
                                              batch_size=batch_size,
                                              class_mode='binary')

test_generator = datagen.flow_from_directory(test_dir,
                                             target_size=(224, 224),
                                             batch_size=batch_size,
                                             class_mode='binary')


## **Product Identification using R-CNN**

In [ ]:
# Use a pre-trained ResNet50 as the base model for Faster R-CNN
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)

model_frcnn = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model_frcnn.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

# Train the model
model_frcnn.fit(train_generator, epochs=5, validation_data=test_generator)


## ***Image Classification using GAN model***

### **import necessary libraries**

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten, Input
from tensorflow.keras.optimizers import Adam
import numpy as np


### **Define GAN architecture for image classification**

In [ ]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(100,))
    x = generator(gan_input)
    gan_output = discriminator(x)
    gan = Model(gan_input, gan_output)
    gan.compile(optimizer='adam', loss='binary_crossentropy')
    return gan


### **Define a simple generator**

In [ ]:
def build_generator(latent_dim, output_shape):
    model = Sequential()
    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(output_shape), activation='sigmoid'))
    model.add(Reshape(output_shape))
    return model


### **Define a simple discriminator**

In [ ]:
def build_discriminator(input_shape):
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model


### **Build and compile the generator and discriminator**

In [ ]:
latent_dim = 100
img_shape = (64, 64, 3)

generator = build_generator(latent_dim, img_shape)
discriminator = build_discriminator(img_shape)
gan = build_gan(generator, discriminator)


### **Train the GAN**

In [ ]:
from tqdm import tqdm
dataset = train_generator

def train_gan(dataset, gan, epochs=10, batch_size=32, latent_dim=100):
    half_batch = batch_size // 2

    for epoch in range(epochs):
        # Train discriminator
        for _ in tqdm(range(len(dataset)//batch_size), desc=f"Epoch {epoch+1}/{epochs} - Discriminator"):
            real_images, _ = dataset.next()
            noise = np.random.normal(0, 1, (half_batch, latent_dim))
            generated_images = gan.generator.predict(noise)

            real_labels = np.ones((half_batch, 1))
            fake_labels = np.zeros((half_batch, 1))

            d_loss_real = gan.discriminator.train_on_batch(real_images, real_labels)
            d_loss_fake = gan.discriminator.train_on_batch(generated_images, fake_labels)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train generator
        for _ in tqdm(range(len(dataset)//batch_size), desc=f"Epoch {epoch+1}/{epochs} - Generator"):
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            valid_labels = np.ones((batch_size, 1))

            g_loss = gan.combined.train_on_batch(noise, valid_labels)

        # Print progress
        print(f"Epoch {epoch+1}/{epochs} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")


### **Extract features from the trained GAN model**

In [ ]:
def extract_gan_features(generator, data_generator, num_samples=1000):
    features = []
    for _ in range(num_samples):
        batch, _ = data_generator.next()
        generated_images = generator.predict(np.random.normal(0, 1, (batch_size, latent_dim)))
        features.extend(generated_images)
    return np.array(features)

### **Train the GAN model and extract features**

In [ ]:
#gan.fit()
train_gan_features = extract_gan_features(generator, train_generator)
test_gan_features = extract_gan_features(generator, test_generator)

### **Define a basic CNN classifier for image classification**

In [ ]:
classifier = Sequential()
classifier.add(Conv2D(32, kernel_size=(3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())
classifier.add(Dense(128, activation='relu'))
classifier.add(Dense(1, activation='sigmoid'))

classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


### **Train the classifier**

In [ ]:
classifier.fit(train_gan_features, train_generator.classes, epochs=5, batch_size=batch_size, validation_data=(test_gan_features, test_generator.classes))

### **Save the trained GAN and classifier models**

In [ ]:
gan.save('gan_model.h5')
classifier.save('classifier_model.h5')

## **Integration and Testing**

### **Import necessary libraries**

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import cv2

loaded_gan = load_model('gan_model.h5')
loaded_classifier = load_model('classifier_model.h5')

### **Product Identification**

Use the trained Faster R-CNN model to identify products in an image

In [ ]:
# Function to perform object detection using Faster R-CNN

def detect_objects(image_path):
    # Load the image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform object detection
    results = model_frcnn.predict(np.expand_dims(image_rgb, axis=0))

    # Extract bounding boxes and class labels
    boxes = results['detection_boxes'][0].numpy()
    scores = results['detection_scores'][0].numpy()
    classes = results['detection_classes'][0].numpy().astype(np.int)

    # Filter out low-confidence detections (adjust threshold as needed)
    threshold = 0.5
    selected_boxes = boxes[scores >= threshold]
    selected_classes = classes[scores >= threshold]

    return selected_boxes, selected_classes

# Example usage:
image_path = '/path/to/test/image.jpg'
detected_boxes, detected_classes = detect_objects(image_path)


### **Crop and preprocess the identified products**

In [ ]:
def crop_and_preprocess(image, boxes):
    cropped_products = []
    for box in boxes:
        ymin, xmin, ymax, xmax = box
        ymin, xmin, ymax, xmax = int(ymin * image.shape[0]), int(xmin * image.shape[1]), int(ymax * image.shape[0]), int(xmax * image.shape[1])

        # Crop the product from the image
        cropped_product = image[ymin:ymax, xmin:xmax]

        # Resize the cropped product to the required input size for GAN
        cropped_product = cv2.resize(cropped_product, (224, 224))

        # Preprocess the image (adjust as needed)
        cropped_product = cropped_product / 255.0  # Normalize to [0, 1]

        cropped_products.append(cropped_product)

    return cropped_products

# Example usage:
cropped_products = crop_and_preprocess(cv2.imread(image_path), detected_boxes)


### **Binary Image Classification**

Use the GAN model to classify each product image as genuine or fake

In [ ]:
# Now you can use the GAN and classifier models to classify each cropped product
for product_image in cropped_products:
    # Use the GAN model to classify each product image as genuine or fake
    features = extract_gan_features(generator, np.expand_dims(product_image, axis=0))
    classification = classifier.predict(features)

    # Print the classification result (you may want to store or use this information as needed)
    print(f'Product Classification: {classification}')